# Selección de características

Una vez extraidas las características hay que analizarlas y seleccionar aquellas que consideremos útiles

## Imports necesarios

In [1]:
import os
import pickle
from collections import defaultdict, Counter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from openai import OpenAI
from dotenv import load_dotenv

c:\Users\evano\Documents\GitHub\TFG\tfg-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Funciones que se van a usar

Función para importar estructuras de datos de ficheros .pkl

In [2]:
def loadCharacteristics(path):
    with open(path, "rb") as f:
        return pickle.load(f)

Función para guardar datos como .pkl

In [3]:
def saveCharacteristics(path, data):
    with open(path, "wb") as file:
        pickle.dump(data, file)

Función para guardar un diccionario de características como un fichero .txt

In [4]:
def saveCharacteristicsAsTxt(path, dataCounter):
    with open(path, "w", encoding = "utf-8") as file:
        for key in dataCounter:
            file.write(str(key) + ": ")

            for subkey in dataCounter[key]:
                file.write("(" + str(subkey) + ", " + str(dataCounter[key][subkey]) + ")" )
            
            file.write("\n")

## Selección de sustantivos

In [5]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/3. Filtered Antonyms"

nouns = {}

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".pkl"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = loadCharacteristics(inputPath)

    for noun in characteristics.keys():
        if noun in nouns:
            nouns[noun] += 1
        else:
            nouns[noun] = 1

In [6]:
nounsOrdenado = dict(sorted(nouns.items(), key=lambda x: x[1], reverse=True))
nounsOrdenado

{'estacion': 245,
 'metro': 171,
 'escalera': 158,
 'movilidad': 147,
 'acceso': 138,
 'servicio': 106,
 'linea': 103,
 'cobertura': 74,
 'zona': 73,
 'maquina': 65,
 'conexion': 63,
 'salida': 60,
 'senalizacion': 48,
 'personal': 45,
 'instalacion': 45,
 'frecuencia': 40,
 'ascensor': 38,
 'tren': 37,
 'gente': 36,
 'estado': 35,
 'informacion': 34,
 'experiencia': 34,
 'transporte': 33,
 'persona': 32,
 'limpieza': 32,
 'diseno': 30,
 'autobus': 30,
 'aire': 30,
 'vestibulo': 30,
 'anden': 28,
 'horario': 26,
 'barrio': 26,
 'calle': 26,
 'hora': 26,
 'verde': 26,
 'atencion': 25,
 'comunicacion': 24,
 'problema': 23,
 'red': 23,
 'seguridad': 22,
 'super': 22,
 'plaza': 22,
 'opinion': 21,
 'bano': 20,
 'dia': 20,
 'centro': 20,
 'tiempo': 20,
 'panel': 19,
 'usuarios': 18,
 'limpio': 18,
 'puerta': 17,
 'intercambiador': 17,
 'senora': 17,
 'maquinas': 17,
 'abono': 17,
 'entorno': 16,
 'moderna': 16,
 'olor': 15,
 'cartel': 15,
 'tarjeta': 15,
 'falta': 15,
 'forma': 14,
 'arquit

In [7]:
len(nouns.keys())

1438

Como se puede ver, hay muchos sustantivos, pero la mayoría no son relevantes. Además, aquellos que no son representatitvos no tienen mucha frecuencia (número de estaciones en las que aparece).

Se va a guardar esta información en un fichero llamado nouns.txt

In [8]:
with open(os.path.join(inputFolderPath, "nouns.txt"), "w", encoding = "utf-8") as file:
    for noun in nouns.keys():
        file.write(noun + ": " + str(nouns[noun]) + "\n")

In [9]:
limit = 30
counter = 0

for noun in nouns.keys():
    if nouns[noun] > limit:
        counter += 1

print("sustantivos con una frecuencia mayor a " + str(limit) + ": " + str(counter))

sustantivos con una frecuencia mayor a 30: 25


Se van a ver cuales son esos sustantivos

In [10]:
filteredNouns = {}

for noun in nouns.keys():
    if nouns[noun] > limit:
        filteredNouns[noun] = nouns[noun]

filteredNouns

{'acceso': 138,
 'movilidad': 147,
 'cobertura': 74,
 'estacion': 245,
 'senalizacion': 48,
 'maquina': 65,
 'escalera': 158,
 'salida': 60,
 'metro': 171,
 'servicio': 106,
 'personal': 45,
 'informacion': 34,
 'conexion': 63,
 'zona': 73,
 'instalacion': 45,
 'ascensor': 38,
 'estado': 35,
 'tren': 37,
 'linea': 103,
 'gente': 36,
 'transporte': 33,
 'experiencia': 34,
 'persona': 32,
 'limpieza': 32,
 'frecuencia': 40}

Únicamente vamos a seleccionar los sustantivos que aparezcan en más de 30 estaciones 

In [11]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/3. Filtered Antonyms"
outputFolderPath = "../1. Data/6. Reviews Characteristics/4. Filtered Nouns"

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".pkl"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = loadCharacteristics(inputPath)

    # Filter the selected nouns 
    finalCharacteristics = {}
    for noun in characteristics.keys():
        if noun in filteredNouns.keys():
            finalCharacteristics[noun] = characteristics[noun]
    
    # Save the data
    station = station[:-4]

    outputPathPkl = os.path.join(outputFolderPath, station + ".pkl")
    outputPathTxt = os.path.join(outputFolderPath, station + ".txt")

    saveCharacteristics(outputPathPkl, Counter(finalCharacteristics))
    saveCharacteristicsAsTxt(outputPathTxt, Counter(finalCharacteristics))
    
    

## Eliminación de intensificadores y negadores

Hasta el momento hemos trabajado con los textos originales de las reviews. Pero llegados a este momento, lo más adecuado es usar los lemas de las caracterísitcas.

El primer paso es eliminar los intensificadores y los negadores (mucho, bien, poco, nunca, no ...)

In [5]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/4. Filtered Nouns"

adjetives = {}

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".pkl"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = loadCharacteristics(inputPath)

    for adjs in characteristics.values():
        for adj, freq in adjs.items():
            if adj in adjetives:
                adjetives[adj] += freq
            else:
                adjetives[adj] = freq

De todas las características vamos a escoger aquellas que contengan más de una palabra

In [6]:
moreThanOneWordKeys = {}

for key in adjetives.keys():
    if len(key.split()) > 1:
        moreThanOneWordKeys[key] = adjetives[key]

moreThanOneWordKeys

{'mucho tranquilo': 22,
 'siempre tanto limpio': 1,
 'mucho importante': 2,
 'mucho amenudo': 1,
 'no encontrar': 3,
 'mucho mal': 12,
 'poco práctico': 4,
 'mucho moderno': 31,
 'mucho concurrido': 54,
 'mucho funcional': 9,
 'mucho fácil': 16,
 'bien comunicado': 149,
 'mucho lejos': 2,
 'relativamente vacío': 1,
 'también limpio': 1,
 'mucho desarrollado': 2,
 'mucho buen': 84,
 'quizá demasiado grande': 12,
 'no necesario': 5,
 'mucho rico': 4,
 'totalmente accesible': 6,
 'perfectamente señalizado': 8,
 'bien organizado': 24,
 'evidentemente tirar': 1,
 'mas nuevo': 1,
 'bien conectado': 7,
 'no mucho generoso': 1,
 'más cercano': 24,
 'no caótico': 1,
 'mucho amable': 36,
 'mucho eficiente': 6,
 'siempre presente': 1,
 'lamentablemente no actualizado': 1,
 'mucho limpio': 105,
 'demasiado complicado': 1,
 'más simple': 2,
 'mucho buena': 18,
 'mas ver': 1,
 'después vuelto': 1,
 'bastante amplio': 34,
 'bien cuidado': 52,
 'aquí pasar': 1,
 'mucho mala': 4,
 'cerca situada': 2,
 

In [8]:
len(moreThanOneWordKeys)

751

Se guardan estos datos en un txt

In [ ]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/5. Simplified Characteristics"

with open(os.path.join(inputFolderPath, "moreThanOneWordKeys.txt"), "w", encoding = "utf-8") as file:
    for adj in moreThanOneWordKeys.keys():
        file.write(adj + ": \n")

Se procesan todas las características con mas de una palabra y se genera un diccionario con sus equivalentes (con una sola palabra)

In [ ]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/5. Simplified Characteristics"

equivalent = {}
with open(os.path.join(inputFolderPath, "moreThanOneWordKeysValues.txt"), encoding = "utf-8") as file:
    
    for line in file:
        elems = line.split(":")
        elems[1] = elems[1].strip()
        equivalent[elems[0]] = elems[1]

In [25]:
equivalent

{'mucho tranquilo': 'tranquilo',
 'siempre tanto limpio': 'limpio',
 'mucho importante': 'importante',
 'mucho amenudo': 'amenudo',
 'no encontrar': 'perder',
 'mucho mal': 'mal',
 'poco práctico': 'inutil',
 'mucho moderno': 'moderno',
 'mucho concurrido': 'concurrido',
 'mucho funcional': 'funcional',
 'mucho fácil': 'facil',
 'bien comunicado': 'comunicado',
 'mucho lejos': 'lejos',
 'relativamente vacío': 'vacio',
 'también limpio': 'limpio',
 'mucho desarrollado': 'desarrollado',
 'mucho buen': 'bien',
 'quizá demasiado grande': 'grande',
 'no necesario': 'innecesario',
 'mucho rico': 'rico',
 'totalmente accesible': 'accesible',
 'perfectamente señalizado': 'señalizado',
 'bien organizado': 'organizado',
 'evidentemente tirar': 'tirar',
 'mas nuevo': 'nuevo',
 'bien conectado': 'conectado',
 'no mucho generoso': 'tacaño',
 'más cercano': 'proximo',
 'no caótico': 'organizado',
 'mucho amable': 'amable',
 'mucho eficiente': 'eficiente',
 'siempre presente': 'presente',
 'lamentabl

In [26]:
len(equivalent.keys())

751

Una vez obtenido este diccionario, se van a sustituir sus claves por sus valores

In [31]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/4. Filtered Nouns"
ouputFolderPath = "../1. Data/6. Reviews Characteristics/5. Simplified Characteristics"

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".pkl"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = loadCharacteristics(inputPath)

    finalCharacteristics = {}
    for noun in characteristics.keys():
        finalAdjectives = {}

        for adj in characteristics[noun].keys():

            if adj in equivalent.keys():
                if equivalent[adj] in finalAdjectives.keys():
                    finalAdjectives[equivalent[adj]] += characteristics[noun][adj]
                else:
                    finalAdjectives[equivalent[adj]] = characteristics[noun][adj]
            else:
                if adj in finalAdjectives.keys():
                    finalAdjectives[adj] += characteristics[noun][adj]
                else:
                    finalAdjectives[adj] = characteristics[noun][adj]

        finalCharacteristics[noun] = Counter(finalAdjectives)
    
    finalCharacteristics = Counter(finalCharacteristics)

    station = station[:-4]

    finalPath = os.path.join(ouputFolderPath, station)
    saveCharacteristics(finalPath + ".pkl", finalCharacteristics)
    saveCharacteristicsAsTxt(finalPath + ".txt", finalCharacteristics)

## Lematización

Para una gestión más facil de las caracaterísticas conviene lematizar

### Mediante llamadas a API de OpenAI

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

def generateText(model, systemMessageContent, prompt):
    systemMessage = {"role" : "system", "content" : systemMessageContent}
    userMessage = {"role" : "user", "content" : prompt}
    messages = [systemMessage, userMessage]

    response = client.chat.completions.create(
        model = model,
        messages = messages
    )

    
    return response.choices[0].message.content

def lemmatize(text):
    model = "o4-mini"
    systemMessageContent = "Eres un modelo de lenguaje especializado en lingüística del español. Tu única tarea es, dada una palabra en castellano, devolver exclusivamente su lema (forma canónica), sin añadir nada más: ni explicaciones, ni etiquetas, ni ejemplos adicionales."
    prompt = """Aquí tienes unos ejemplos de cómo debes formatear la respuesta:

    Ejemplos:
    input: bien
    output: bien

    input: facilisimo
    output: facil

    input: superfacil
    output: facil

    Ahora realiza la tarea:
    input: {input_text}
    output:"""

    return generateText(model, systemMessageContent, prompt.format(input_text = text))

Vamos a probar varios casos y escoger que método es mejor

In [33]:
lemmatize("superbien")

'bien'

In [34]:
lemmatize("estupendisimo")

'estupendo'

In [35]:
lemmatize("intransitado")

'transitar'

In [36]:
lemmatize("concurrido")

'concurrir'

Como se puede ver, es inaceptable que intransitado lo cambie a transitar, ya que son opuestos.

### Con Spacy

In [5]:
nlp = spacy.load("es_core_news_md")

def lemmatize(text):
    doc = nlp(text)
    lemmas = [tok.lemma_.lower() for tok in doc if not tok.is_punct]

    return " ".join(lemmas)

In [43]:
lemmatize("superbien")

'superbien'

In [54]:
lemmatize("estupendísimo")

'estupendísimo'

In [45]:
lemmatize("intransitado")

'intransitado'

In [55]:
lemmatize("limpia")

'limpio'

In [53]:
lemmatize("bonitos")

'bonito'

Este método es más estable, se ajusta mejor a lo que requerimos

In [6]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/5. Simplified Characteristics"
ouputFolderPath = "../1. Data/6. Reviews Characteristics/6. Lemmatization"

adjCorpus = {}
lemmatized = {}

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".pkl"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = loadCharacteristics(inputPath)

    # Lemmatize and extract the adjective corpus
    finalCharacteristics = {}
    for noun in characteristics.keys():
        finalAdjectives = {}

        for adj in characteristics[noun].keys():
            
            # To avoid repetition
            if adj in lemmatized:
                lemma = lemmatized[adj]
            else:
                lemma = lemmatize(adj)
                lemmatized[adj] = lemma

            # Update the dictionary
            if lemma in finalAdjectives.keys():
                finalAdjectives[lemma] += characteristics[noun][adj]
            else:
                finalAdjectives[lemma] = characteristics[noun][adj]
            
            # Add to the corpus
            if lemma in adjCorpus.keys():
                adjCorpus[lemma] += characteristics[noun][adj]
            else:
                adjCorpus[lemma] = characteristics[noun][adj]

        finalCharacteristics[noun] = Counter(finalAdjectives)
    
    finalCharacteristics = Counter(finalCharacteristics)

    station = station[:-4]

    finalPath = os.path.join(ouputFolderPath, station)
    saveCharacteristics(finalPath + ".pkl", finalCharacteristics)
    saveCharacteristicsAsTxt(finalPath + ".txt", finalCharacteristics)

# Save the corpus information
with open(os.path.join(ouputFolderPath, "adjectivesCorpus.txt"), "w", encoding = "utf-8") as file:
    for adj in adjCorpus.keys():
        file.write(adj + ": " + str(adjCorpus[adj]) + "\n")

    

In [7]:
len(adjCorpus)

1170

## Selección de caracteristicas

Se procede a repetir el mismo proceso pero con cada una de las caracteristicas de los sustantivos

In [ ]:
inputFolderPath = "../1. Data/6. Reviews Characteristics/6. Lemmatization"

adjetives = {}

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".pkl"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = loadCharacteristics(inputPath)

    for adjs in characteristics.values():
        for adj, freq in adjs.items():
            if adj in adjetives:
                adjetives[adj] += freq
            else:
                adjetives[adj] = freq

In [ ]:
adjetivesSorted = dict(sorted(adjetives.items(), key=lambda x: x[1], reverse=True))
adjetivesSorted

In [7]:
len(adjetivesSorted.keys())

1170

In [23]:
adjetivesSorted

{'buen': 664,
 'limpio': 616,
 'bien': 524,
 'mecánico': 366,
 'reducido': 290,
 'grande': 249,
 'moderno': 245,
 'importante': 222,
 'amplio': 214,
 'bonito': 212,
 'accesible': 192,
 'comunicado': 169,
 'situado': 149,
 'ubicado': 146,
 'excelente': 135,
 'antiguo': 134,
 'normal': 120,
 'público': 119,
 'parado': 113,
 'nuevo': 109,
 'fácil': 106,
 'cuidado': 106,
 'mejor': 98,
 'gran': 96,
 'tranquilo': 88,
 'señalizar': 88,
 'concurrido': 82,
 'ligero': 72,
 'móvil': 71,
 'ascensor': 70,
 'intercambiador': 68,
 'perfecto': 67,
 'adaptado': 66,
 'equipado': 65,
 'amable': 64,
 'cómodo': 64,
 'pequeño': 60,
 'claro': 59,
 'bueno': 54,
 'cerca': 53,
 'seguro': 52,
 'discapacitado': 50,
 'único': 50,
 'organizado': 47,
 'sucio': 47,
 'rápido': 46,
 'lindo': 46,
 'expendedor': 45,
 'comodo': 43,
 'enorme': 43,
 'facil': 42,
 'ordenado': 41,
 'malo': 41,
 'conectado': 40,
 'genial': 40,
 'junto': 40,
 'disponible': 39,
 'inadaptado': 39,
 'comercial': 39,
 'residencial': 39,
 'peor': 38

Al igual que con los sustantivos, hay muchos adjetivos, pero la mayoría no son relevantes. Además, aquellos que no son representatitvos no tienen mucha frecuencia. Por lo que vamos a filtrarlos por un cierto limite.

In [24]:
aux = {}
limit = 4

for adj in adjetives.keys():
    if adjetives[adj] >= limit:
        aux[adj] = adjetives[adj]

filteredAdjetives = dict(sorted(aux.items(), key=lambda x: x[1], reverse=True))
filteredAdjetives

{'buen': 664,
 'limpio': 616,
 'bien': 524,
 'mecánico': 366,
 'reducido': 290,
 'grande': 249,
 'moderno': 245,
 'importante': 222,
 'amplio': 214,
 'bonito': 212,
 'accesible': 192,
 'comunicado': 169,
 'situado': 149,
 'ubicado': 146,
 'excelente': 135,
 'antiguo': 134,
 'normal': 120,
 'público': 119,
 'parado': 113,
 'nuevo': 109,
 'fácil': 106,
 'cuidado': 106,
 'mejor': 98,
 'gran': 96,
 'tranquilo': 88,
 'señalizar': 88,
 'concurrido': 82,
 'ligero': 72,
 'móvil': 71,
 'ascensor': 70,
 'intercambiador': 68,
 'perfecto': 67,
 'adaptado': 66,
 'equipado': 65,
 'amable': 64,
 'cómodo': 64,
 'pequeño': 60,
 'claro': 59,
 'bueno': 54,
 'cerca': 53,
 'seguro': 52,
 'discapacitado': 50,
 'único': 50,
 'organizado': 47,
 'sucio': 47,
 'rápido': 46,
 'lindo': 46,
 'expendedor': 45,
 'comodo': 43,
 'enorme': 43,
 'facil': 42,
 'ordenado': 41,
 'malo': 41,
 'conectado': 40,
 'genial': 40,
 'junto': 40,
 'disponible': 39,
 'inadaptado': 39,
 'comercial': 39,
 'residencial': 39,
 'peor': 38

In [ ]:
len(filteredAdjetives.keys())

with open(os.path.join(outputFolderPath, "adjectivesFilteredOrdered.txt"), "w", encoding = "utf-8") as file:
    for adj in filteredAdjetives.keys():
        file.write(adj + ": \n")

## Sinonimos y eliminación de características poco representativas

Aquellas características con menos de 4 apariciones se van a ignorar. Además, se van a sustituir por sus sinónimos aquellas que si se tendrán en cuenta cuando sea posible

In [ ]:
ouputFolderPath = "../1. Data/6. Reviews Characteristics/7. Filtered Characteristics"
inputFolderPath = "../1. Data/6. Reviews Characteristics/6. Lemmatization"

# Load the dictionary of synonyms
synonyms = {}
with open(os.path.join(outputFolderPath, "synonymsDict.txt"), "w", encoding = "utf-8") as file:
    for line in file:
        elems = line.split(":")
        elems[1] = elems[1].strip()
    
    synonyms[elems[0]] = elems[1]


adjCorpus = {}
lemmatized = {}

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".pkl"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    characteristics = loadCharacteristics(inputPath)

    # Lemmatize and extract the adjective corpus
    finalCharacteristics = {}
    for noun in characteristics.keys():
        finalAdjectives = {}

        for adj in characteristics[noun].keys():
            
           if adj in synonyms.keys():
               synonym = synonyms[adj]

                if synonym in finalAdjectives:
                   finalAdjectives[synonym] += characteristics[adj]
                else:
                   finalAdjectives[synonym] = characteristics[adj]
                   

        finalCharacteristics[noun] = Counter(finalAdjectives)
    
    finalCharacteristics = Counter(finalCharacteristics)

    station = station[:-4]

    finalPath = os.path.join(ouputFolderPath, station)
    saveCharacteristics(finalPath + ".pkl", finalCharacteristics)
    saveCharacteristicsAsTxt(finalPath + ".txt", finalCharacteristics)

# Save the corpus information
with open(os.path.join(ouputFolderPath, "adjectivesCorpus.txt"), "w", encoding = "utf-8") as file:
    for adj in adjCorpus.keys():
        file.write(adj + ": " + str(adjCorpus[adj]) + "\n")



# Save the corpus information
with open(os.path.join(ouputFolderPath, "synonymsDict.txt"), "w", encoding = "utf-8") as file:
    for adj in synonyms.keys():
        file.write(adj + ": " + synonyms[adj] + "\n")


IndentationError: expected an indented block after 'if' statement on line 33 (4009181377.py, line 35)